In [1]:
import pandas as pd
import numpy as np
import dgl
import createGraph
from importlib import reload
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import architecture
import training_loop


TypeError: throughData() missing 1 required positional argument: 'y'

In [ ]:
# Load Data
df = pd.read_csv('duke_vital_model_imputed.csv')
X = df.drop(labels=["PostCond"], axis=1).to_numpy()
y = df["PostCond"].to_numpy()

In [ ]:
# impute missing values - process will be made a little more complex later when we impute train and test vals separately; for now we can do it like this
imputer = IterativeImputer(random_state=42)
X = imputer.fit_transform(X)

# convert imputed ndarrays back into a Dataframe
imputed_X_df = pd.DataFrame(data=X, columns=df.drop(labels=["PostCond"], axis=1).columns)
y_df = pd.DataFrame(data=y, columns=["PostCond"])
imputed_df = pd.concat([imputed_X_df, y_df], axis=1)

In [ ]:
# load_graphs - figure out how to do this
g, y, node_batches = createGraph.throughData(imputed_df)

In [ ]:
# split into train/val sets and test sets
skf = StratifiedKFold(n_splits = 10, shuffle=True, random_state=42)
indices = skf.split(np.zeros(len(y)), y)
train_val_mask = dict()
test_mask = dict()
cnt = 0
for train_i, test_i in indices:
    train_val_mask[cnt] = train_i
    test_mask[cnt] = test_i
    cnt += 1

In [ ]:
# split train/val sets into train sets and val sets
train_mask = dict()
val_mask = dict()

for i in range(0, 10):
    train_mask[i], val_mask[i] = train_test_split(train_val_mask[i], y[train_val_mask[i]], test_size=0.1)

In [ ]:
model = architecture.GCN(g['feats'].shape(-1), 30, 30)

training_loop.train(g=g, node_batches=node_batches, labels=y, train_mask=train_mask[0], val_mask=val_mask[0], validate=True, test=False)